In [1]:
import tensorflow as tf

print(tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

2.1.0


In [2]:
class Inception(tf.keras.layers.Layer):
    def __init__(self,c1,c2,c3,c4):
        super().__init__()
        # 1
        self.p1_1 = tf.keras.layers.Conv2D(c1,kernel_size=1,activation='relu',padding='same')
        # 2
        self.p2_1 = tf.keras.layers.Conv2D(c2[0],kernel_size=1,activation='relu',padding='same')
        self.p2_2 = tf.keras.layers.Conv2D(c2[1],kernel_size=3,activation='relu',padding='same')
        # 3
        self.p3_1 =  tf.keras.layers.Conv2D(c3[0],kernel_size=1,activation='relu',padding='same')
        self.p3_2 = tf.keras.layers.Conv2D(c3[1],kernel_size=5,activation='relu',padding='same')
        # 4
        self.p4_1 = tf.keras.layers.MaxPool2D(pool_size=3,padding='same',strides=1)
        self.p4_2 = tf.keras.layers.Conv2D(c4,kernel_size=1,padding='same',activation='relu')
        
        
    def call(self,x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return tf.concat([p1,p2,p3,p4],axis=-1)

In [3]:
b1 = tf.keras.models.Sequential()
b1.add(tf.keras.layers.Conv2D(64,kernel_size=7,strides=2,padding='same',activation='relu'))
b1.add(tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))

In [4]:
b2 = tf.keras.models.Sequential()
b2.add(tf.keras.layers.Conv2D(64,kernel_size=1,padding='same',activation='relu'))
b2.add(tf.keras.layers.Conv2D(192,kernel_size=3,padding='same',activation='relu'))
b2.add(tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))

In [5]:
b3 = tf.keras.models.Sequential()
b3.add(Inception(64,(96,128),(16,32),32))
b3.add(Inception(128,(128,192),(32,96),64))
b3.add(tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))

In [6]:
b4 = tf.keras.models.Sequential()
b4.add(Inception(192,(96,288),(16,48),64))
b4.add(Inception(160,(112,224),(24,64),64))
b4.add(Inception(128,(128,256),(16,64),64))
b4.add(Inception(112,(144,288),(16,64),64))
b4.add(Inception(256,(160,320),(32,128),128))
b4.add(tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same'))


In [7]:
b5 = tf.keras.models.Sequential()
b5.add(Inception(256,(160,320),(32,128),128))
b5.add(Inception(384,(192,384),(48,128),128))
b5.add(tf.keras.layers.GlobalAvgPool2D())

In [8]:
net = tf.keras.models.Sequential([b1,b2,b3,b4,b5,tf.keras.layers.Dense(10)])

In [9]:
X = tf.random.uniform(shape=(1, 96, 96, 1))
for layer in net.layers:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential output shape:	 (1, 24, 24, 64)
sequential_1 output shape:	 (1, 12, 12, 192)
sequential_2 output shape:	 (1, 6, 6, 480)
sequential_3 output shape:	 (1, 3, 3, 832)
sequential_4 output shape:	 (1, 1024)
dense output shape:	 (1, 10)


In [10]:
import numpy as np

class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images,self.train_labels),(self.test_images,self.test_labels) = fashion_mnist.load_data()
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.,axis=-1)
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.,axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_images = self.test_images.astype(np.int32)
        self.num_train,self.num_test = self.train_images.shape[0],self.test_images.shape[0]
        
    
    def get_batch_train(self,batch_size):
        index= np.random.randint(0,np.shape(self.train_images)[0],batch_size)
        resized_images = tf.image.resize_with_pad(self.train_images[index],224,224)
        return resized_images.numpy(),self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.test_images[index],224,224)
        return resized_images.numpy(), self.test_labels[index]

batch_size = 128
dataLoader = DataLoader()
x_batch,y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)

x_batch shape: (128, 224, 224, 1) y_batch shape: (128,)


In [11]:
def train_googlenet():
    epoch = 1
    num_iter = dataLoader.num_train//batch_size
    for e in range(epoch):
        for n in range(num_iter):
            x_batch, y_batch = dataLoader.get_batch_train(batch_size)
            net.fit(x_batch, y_batch,verbose=2)
            if n%20 == 0:
                net.save_weights("5.9_googlenet_weights.h5")

optimizer = tf.keras.optimizers.Adam(lr=1e-7)

net.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

train_googlenet()


Train on 128 samples
128/128 - 10s - loss: 6.9405 - accuracy: 0.0547
Train on 128 samples
128/128 - 0s - loss: 6.0856 - accuracy: 0.1094
Train on 128 samples
128/128 - 0s - loss: 5.8489 - accuracy: 0.0938
Train on 128 samples
128/128 - 0s - loss: 5.3333 - accuracy: 0.0781
Train on 128 samples
128/128 - 1s - loss: 7.2317 - accuracy: 0.0469
Train on 128 samples
128/128 - 1s - loss: 5.9399 - accuracy: 0.0547
Train on 128 samples
128/128 - 1s - loss: 6.3587 - accuracy: 0.0781
Train on 128 samples
128/128 - 1s - loss: 5.8047 - accuracy: 0.1016
Train on 128 samples
128/128 - 1s - loss: 6.1170 - accuracy: 0.0391
Train on 128 samples
128/128 - 1s - loss: 6.6031 - accuracy: 0.0625
Train on 128 samples
128/128 - 1s - loss: 5.7512 - accuracy: 0.0859
Train on 128 samples
128/128 - 1s - loss: 6.3204 - accuracy: 0.1016
Train on 128 samples
128/128 - 1s - loss: 6.8950 - accuracy: 0.0859
Train on 128 samples
128/128 - 1s - loss: 6.4142 - accuracy: 0.0781
Train on 128 samples
128/128 - 1s - loss: 6.347

In [12]:
x_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(x_test, y_test, verbose=1)

2000/2000 [==============================] - 6s 3ms/sample - loss: 5.8774 - accuracy: 0.1050

[5.8773937454223635, 0.105]